<a href="https://colab.research.google.com/github/l1kemood/Demo/blob/main/mnist_%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
from torchvision import datasets,transforms # Corrected the module name from 'torchvison' to 'torchvision'
from torch.utils.data import DataLoader
import torch

In [ ]:
train_dataset=datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.6MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 476kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.38MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.71MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#Batch size
batch_size=64
#Loaded in the training set
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
#Loaded in the testing set
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
for i,data in enumerate(train_loader):
  inputs,labels=data
  print(inputs.shape)
  print(labels.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [ ]:
#Define network structure
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Sequential(nn.Conv2d(1,32,5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
    self.conv2 = nn.Sequential(nn.Conv2d(32,64,5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
    self.fc1 = nn.Sequential(nn.Linear(64*7*7,1000),nn.Dropout(p=0.5),nn.ReLU())
    self.fc2 = nn.Sequential(nn.Linear(1000,10),nn.Softmax(dim=1))

  def forward(self,x):
    # [64, 1, 28, 28]
    x = self.conv1(x)
    x = self.conv2(x)
    #(64,64,7,7)
    x = x.view(x.size()[0],-1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [ ]:
LR = 0.001
#Define the model
model = Net()
#Define cost function
mse_loss = nn.CrossEntropyLoss()
#Define optimizer,Set L2 regularization
optimizer = optim.Adam(model.parameters(),LR)

In [ ]:
def train():
  #Training state
  model.train()
  for i,data in enumerate(train_loader):
    #Get a batch of data and labels
    inputs,labels = data
    #Obtain model prediction results (64,10)
    out = model(inputs)
    #Cross entropy cost function out(batch,C),labels (batch)
    loss = mse_loss(out,labels)
    #Gradient clearing
    optimizer.zero_grad()
    #Calculated gradient
    loss.backward()
    #Update parameters
    optimizer.step()

def test():
  #Testing state
  model.eval()
  correct = 0
  for i,data in enumerate(test_loader):
    #Get a batch of data and labels
    inputs,labels = data
    #The model prediction results were obtained(64,10)
    out = model(inputs)
    #Get the maximum value and the location of the maximum value
    _,predicted = torch.max(out,1)
    #Predict the correct amount
    correct += (predicted == labels).sum()
  print("Test acc:{0}".format(correct.item()/len(test_dataset)))

  correct = 0
  for i,data in enumerate(train_loader):
    #Get a batch of data and labels
    inputs,labels = data
    #The model prediction results were obtained(64,10)
    out = model(inputs)
    #Get the maximum value and the location of the maximum value
    _,predicted = torch.max(out,1)
    #Predict the correct amount
    correct += (predicted == labels).sum()
  print("Train acc:{0}".format(correct.item()/len(train_dataset)))



In [ ]:
for epoch in range(20):
  print('epoch:',epoch)
  train()
  test()

epoch: 0
Test acc:0.9736
Train acc:0.9705833333333334
epoch: 1
Test acc:0.9853
Train acc:0.9834833333333334
epoch: 2
Test acc:0.9864
Train acc:0.9871833333333333
epoch: 3
Test acc:0.985
Train acc:0.9868333333333333
epoch: 4
Test acc:0.9906
Train acc:0.9905833333333334
epoch: 5
Test acc:0.9861
Train acc:0.9883166666666666
epoch: 6
